<a href="https://colab.research.google.com/github/mraihan19/intent_identify/blob/main/intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mraihan19/intent_identify/blob/main/intent.ipynb)


SyntaxError: ignored

In [1]:
!pip install git+https://github.com/mit-nlp/MITIE.git
!pip install spacy

  Cloning https://github.com/mit-nlp/MITIE.git to /tmp/pip-req-build-mybh04gk
  Running command git clone -q https://github.com/mit-nlp/MITIE.git /tmp/pip-req-build-mybh04gk
  Created wheel for mitie: filename=mitie-0.7.0-cp37-none-any.whl size=418704 sha256=e8e5ed86f9b59c2f204ce67fcedef4ba6b459c6a7a1f4d71182951f11cda49d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-wjkwaklh/wheels/b4/c1/21/8e7e7e14cf3211bf5c73aad0b1d76d1186fbf681f4b9ef6c06
Successfully built mitie


In [2]:
!git clone https://github.com/mit-nlp/MITIE.git

Cloning into 'MITIE'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 33197 (delta 1), reused 3 (delta 0), pack-reused 33190
Receiving objects: 100% (33197/33197), 10.69 MiB | 20.69 MiB/s, done.
Resolving deltas: 100% (23153/23153), done.


In [3]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
!ls

MITIE  sample_data


In [9]:
%%bash
cd MITIE
make MITIE-models

curl -LO https://github.com/mit-nlp/MITIE/releases/download/v0.4/MITIE-models-v0.2.tar.bz2
tar -xjf MITIE-models-v0.2.tar.bz2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   631  100   631    0     0   3252      0 --:--:-- --:--:-- --:--:--  3269
100  415M  100  415M    0     0  61.6M      0  0:00:06  0:00:06 --:--:-- 68.1M


In [10]:
!ls MITIE/MITIE-models/english

binary_relations  ner_model.dat  total_word_feature_extractor.dat


In [11]:
!wget https://www.dropbox.com/s/0abglv8scuorkhs/glove.6B.zip

--2021-04-14 10:42:22--  https://www.dropbox.com/s/0abglv8scuorkhs/glove.6B.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:601a:18::a27d:712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0abglv8scuorkhs/glove.6B.zip [following]
--2021-04-14 10:42:22--  https://www.dropbox.com/s/raw/0abglv8scuorkhs/glove.6B.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc107880ab48266344c2116ccc0c.dl.dropboxusercontent.com/cd/0/inline/BMlpm9NwgJqKDyjwp3I1pYneiOLFPaoPNNWg1cHwMPBmPnS_pBMXZNZQXVIzDyEYAVTRBEquiUtZ626vqJGZYigKvQoBzpIGQzMmEqMJx-S-KmZe84G_obxkqVdDfUX4R_ODdjK8_NcASD6WrGZd59o6/file# [following]
--2021-04-14 10:42:22--  https://uc107880ab48266344c2116ccc0c.dl.dropboxusercontent.com/cd/0/inline/BMlpm9NwgJqKDyjwp3I1pYneiOLFPaoPNNWg1cHwMPBmPnS_pBMXZNZQXVIzDyEYAVTRBEquiUtZ626vqJGZYigKvQ

**Data**

---



Currently it is trained on data for finding Restaurants and foods. With little effort it can be ported to any other type of intention finding



In [12]:
data={
  "greet": {
    "examples" : ["hello","hey there","howdy","hello","hi","hey","hey ho"],
    "centroid" : None
  },
  "inform": {
    "examples" : [
      "i'd like something asian",
      "maybe korean",
      "what mexican options do i have",
      "what italian options do i have",
      "i want korean food",
      "i want german food",
      "i want vegetarian food",
      "i would like chinese food",
      "i would like indian food",
      "what japanese options do i have",
      "korean please",
      "what about indian",
      "i want some vegan food",
      "maybe thai",
      "i'd like something vegetarian",
      "show me french restaurants",
      "show me a cool malaysian spot"
    ],
    "centroid" : None
  },
  "deny": {
    "examples" : [
      "nah",
      "any other places ?",
      "anything else",
      "no thanks"
      "not that one",
      "i do not like that place",
      "something else please",
      "no please show other options"
    ],
    "centroid" : None
  }
}

In [13]:
import sys, os
!unzip glove.6B.zip -d ./glove

Archive:  glove.6B.zip
  inflating: ./glove/glove.6B.50d.txt  
  inflating: ./glove/glove.6B.100d.txt  
  inflating: ./glove/glove.6B.200d.txt  
  inflating: ./glove/glove.6B.300d.txt  


**Network Training**

In [ ]:
from mitie import *

trainer = text_categorizer_trainer("MITIE/MITIE-models/english/total_word_feature_extractor.dat")

 # same as before  - omitted for brevity
training_examples = data

for label in training_examples.keys():
  for text in training_examples[label]["examples"]:
    tokens = tokenize(text)
    trainer.add_labeled_text(tokens,label)
    
trainer.num_threads = 4
cat = trainer.train()

cat.save_to_disk("my_text_categorizer.dat")

**Test**

This model can detect 3 type of intentions:

1.   Inform
2.   Greet &
3.   Deny


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
input_string = "do you have anything else?"
tokens = tokenize(input_string)
predicted_label, _ = cat(tokens)

In [ ]:
print "Intention: ", predicted_label
doc = nlp(u'{}'.format(input_string)) #input_string.encode('utf-8')
for chunk in doc.noun_chunks:
  if chunk.root.dep_ == u'dobj':
    print(chunk.text)